In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.dense1 = nn.Linear(2,5)
#         self.dense2 = nn.Linear(5,5)
#         self.dense3 = nn.Linear(5,1)
        
#     def forward(self, x):
#         x = self.dense1(x)
#         x = F.relu(x)
#         x = self.dense2(x)
#         x = F.relu(x)
#         x = self.dense3(x)
#         x = self.sigmoid(x)
#         return x

# criterion = nn.MSELoss()
# net = Net()
# print(list(net.parameters()))
# optimizer = optim.SGD(net.parameters(),lr=0.01)

# x = net.parameters().__next__()

# print("{}".format(x.data))
# print("{}".format(x.grad))

# input = torch.tensor(torch.rand(1,2), requires_grad=True)
# print(input)

In [15]:
#param initialization
n_input, n_hidden, n_output = 5, 3, 1

## initialize tensor for inputs, and outputs 
x = torch.randn((1, n_input))
y = torch.randn((1, n_output))

## initialize tensor variables for weights 
w1 = torch.randn(n_input, n_hidden) # weight for hidden layer
w2 = torch.randn(n_hidden, n_output) # weight for output layer

## initialize tensor variables for bias terms 
b1 = torch.randn((1, n_hidden)) # bias for hidden layer
b2 = torch.randn((1, n_output)) # bias for output layer

In [ ]:
# forward prop
